# Odległość edycyjna

In [1]:
import numpy as np
from enum import Enum
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish

### Algorytm obliczania odległości edycyjnej

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.

In [2]:
class Step(Enum):
    INIT = 0
    RIGHT = 1
    DOWN = 2
    RIGHT_DOWN = 3

def delta(a,b):
    return a!=b

def edit_distance(x,y,delta):
    edit_table=np.empty((len(x)+1, len(y)+1))
    steps=np.empty((len(x)+1, len(y)+1))
    for i in range(len(x)+1):
        edit_table[i,0]=i
        steps[i,0]=Step.DOWN.value
    for j in range(len(y)+1):
        edit_table[0,j]=j
        steps[0,j]=Step.RIGHT.value
    steps[0,0]=Step.INIT.value
    for i in range(len(x)):
        k=i+1
        for j in range(len(y)):
            l=j+1
            min_val=min(edit_table[k-1,l]+1, edit_table[k,l-1]+1, edit_table[k-1,l-1]+delta(x[i], y[j]))
            edit_table[k,l]=min_val
            if min_val==edit_table[k-1,l]+1:
                steps[k,l]=Step.DOWN.value
            elif min_val==edit_table[k,l-1]+1:
                steps[k,l]=Step.RIGHT.value
            else:
                steps[k,l]=Step.RIGHT_DOWN.value
    return edit_table, steps

In [3]:
print(edit_distance("los", "kloc", delta)[0])

[[0. 1. 2. 3. 4.]
 [1. 1. 1. 2. 3.]
 [2. 2. 2. 1. 2.]
 [3. 3. 3. 2. 2.]]


2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być zrealizowana następująco:
* *k*los (dodanie litery k)
* klo*c* (zamiana s->c)

In [4]:
def get_path(len_x,len_y, steps):
    curr_pointer=[len_x, len_y]
    prev_step=steps[curr_pointer[0], curr_pointer[1]]
    recalled_steps=[]
    while prev_step!=Step.INIT.value:
        if prev_step==Step.RIGHT_DOWN.value:
            curr_pointer[0]-=1
            curr_pointer[1]-=1
        elif prev_step==Step.RIGHT.value:
            curr_pointer[1]-=1
        else:
            curr_pointer[0]-=1
        recalled_steps=[prev_step]+recalled_steps
        prev_step=steps[curr_pointer[0], curr_pointer[1]]
    return recalled_steps

def recall_steps(x,y):
    if(len(y)<len(x)):x,y=y,x
    edit_table, steps=edit_distance(x,y,delta)
    dist=int(edit_table[len(x), len(y)])
    
    recalled_steps=get_path(len(x),len(y), steps)

    print("1.", "START\t", x)
    for i in range(len(recalled_steps)):
        if recalled_steps[i]==Step.RIGHT.value:
            x=x[:i]+y[i]+x[i:]
            print(str(i+2)+".", "ADD\t\t", x[:i]+"*"+x[i]+"*"+x[i+1:])
        elif recalled_steps[i]==Step.DOWN.value:
            print(str(i+2)+".", "REMOVE\t", x[:i]+"*"+x[i+1:])
            x=x[:i]+x[i+1:]
        elif i<len(x) and i<len(y) and x[i]==y[i]:
            print(str(i+2)+".", "OK\t\t", x[:i]+"*"+x[i]+"*"+x[i+1:])
        elif i<len(y):
            x=x[:i]+y[i]+x[i+1:]
            print(str(i+2)+".", "CHANGE\t", x[:i]+"*"+x[i]+"*"+x[i+1:])
    print("\nEDIT DISTANCE =", dist)

3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
* los - kloc
* Łódź - Lodz
* kwintesencja - quintessence
* ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG

In [5]:
recall_steps("kloc", "los")

1. START	 los
2. ADD		 *k*los
3. OK		 k*l*os
4. OK		 kl*o*s
5. CHANGE	 klo*c*

EDIT DISTANCE = 2


In [6]:
recall_steps("Łódź", "Lodz")

1. START	 Łódź
2. CHANGE	 *L*ódź
3. CHANGE	 L*o*dź
4. OK		 Lo*d*ź
5. CHANGE	 Lod*z*

EDIT DISTANCE = 3


In [7]:
recall_steps("kwintesencja", "quintessence")

1. START	 kwintesencja
2. CHANGE	 *q*wintesencja
3. CHANGE	 q*u*intesencja
4. OK		 qu*i*ntesencja
5. OK		 qui*n*tesencja
6. OK		 quin*t*esencja
7. OK		 quint*e*sencja
8. OK		 quinte*s*encja
9. ADD		 quintes*s*encja
10. OK		 quintess*e*ncja
11. OK		 quintesse*n*cja
12. OK		 quintessen*c*ja
13. CHANGE	 quintessenc*e*a
14. REMOVE	 quintessence*

EDIT DISTANCE = 5


In [8]:
recall_steps("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

1. START	 ATGAATCTTACCGCCTCG
2. OK		 *A*TGAATCTTACCGCCTCG
3. OK		 A*T*GAATCTTACCGCCTCG
4. OK		 AT*G*AATCTTACCGCCTCG
5. OK		 ATG*A*ATCTTACCGCCTCG
6. CHANGE	 ATGA*G*TCTTACCGCCTCG
7. CHANGE	 ATGAG*G*CTTACCGCCTCG
8. OK		 ATGAGG*C*TTACCGCCTCG
9. OK		 ATGAGGC*T*TACCGCCTCG
10. ADD		 ATGAGGCT*C*TACCGCCTCG
11. OK		 ATGAGGCTC*T*ACCGCCTCG
12. CHANGE	 ATGAGGCTCT*G*CCGCCTCG
13. ADD		 ATGAGGCTCTG*G*CCGCCTCG
14. OK		 ATGAGGCTCTGG*C*CGCCTCG
15. OK		 ATGAGGCTCTGGC*C*GCCTCG
16. REMOVE	 ATGAGGCTCTGGCC*CCTCG
17. OK		 ATGAGGCTCTGGCCC*C*TCG
18. OK		 ATGAGGCTCTGGCCCC*T*CG
19. CHANGE	 ATGAGGCTCTGGCCCCT*G*G
20. REMOVE	 ATGAGGCTCTGGCCCCTG*

EDIT DISTANCE = 7


### Algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów

4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.

In [9]:
def delta2(x,y):
    if x==y: return 0
    else: return 2

In [10]:
def lcs(x,y):
    return int((len(x)+len(y)-edit_distance(x,y, delta2)[0][len(x)][len(y)])/2)

In [11]:
A="AXXXBXXXCXDE"
B="YYAYBCYYYYYYYYDYYEYYYY"
print(A, "\t", B)
print("Longest common subsequence: ", lcs(A, B))

AXXXBXXXCXDE 	 YYAYBCYYYYYYYYDYYEYYYY
Longest common subsequence:  5


5. Korzystając z gotowego tokenizera (np. spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.

In [12]:
nlp=Polish()
tokenizer=Tokenizer(nlp.vocab)
with open("romeo-i-julia-700.txt", "r", encoding="utf-8") as f:
    text = f.read()
tokens = tokenizer(text)

6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

In [13]:
def del_tok(tokens, percent=3):
    result = []
    for token in tokens:
        if np.random.random() > percent/100:
            result.append(token)
    return result

In [14]:
tokens1=del_tok(tokens)
tokens2=del_tok(tokens)

In [15]:
with open("text1.txt", "w", encoding="utf-8") as f:
    for token in tokens1:
        f.write(token.text_with_ws)
with open("text2.txt", "w", encoding="utf-8") as f:
    for token in tokens2:
        f.write(token.text_with_ws)

7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów. Traktujemy teraz token (wyraz) jako podstawową, niepodzielną jednostkę ciągu.

In [16]:
print("Text1 len: ", len(tokens1))
print("Text2 len: ", len(tokens2))
print("Longest common subsequence: ", lcs(tokens1, tokens2))

Text1 len:  2199
Text2 len:  2203
Longest common subsequence:  2133


8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku. Traktujemy teraz całą linię jako podstawową, niepodzielną jednostkę ciągu.

In [17]:
def diff(x,y, unit_name="FILE", element_name="LINE"):
    if(len(y)<len(x)):x,y=y,x
        
    edit_di, steps=edit_distance(x,y, delta2)
    
    recalled_steps=get_path(len(x),len(y), steps)
    
    currx_i=0
    curry_i=0
    
    for i in range(len(recalled_steps)):
        if recalled_steps[i]==Step.RIGHT_DOWN.value:
            currx_i+=1
            curry_i+=1
        elif recalled_steps[i]==Step.DOWN.value:
            print(unit_name, "1 |", element_name, currx_i, "|", x[currx_i])
            currx_i+=1
        else:
            print(unit_name, "2 |", element_name, curry_i, "|", y[curry_i])
            curry_i+=1

In [18]:
diff("XABCDEYF", "AYBCZDEFZ", "WORD", "LETTER")

WORD 1 | LETTER 0 | X
WORD 2 | LETTER 1 | Y
WORD 2 | LETTER 4 | Z
WORD 1 | LETTER 6 | Y
WORD 2 | LETTER 8 | Z


9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [19]:
with open("text1.txt", "r", encoding="utf-8") as f:
        text1 = f.read()

with open("text2.txt", "r", encoding="utf-8") as f:
    text2 = f.read()

text1 = text1.split('\n')
text2 = text2.split('\n')

In [20]:
diff(text1,text2)

FILE 2 | LINE 0 | William Romeo i Julia
FILE 1 | LINE 0 | William Shakespeare
FILE 1 | LINE 1 | 
FILE 1 | LINE 2 | Romeo i Julia
FILE 2 | LINE 20 |  * ABRAHAM — służący 
FILE 1 | LINE 22 |  * ABRAHAM — służący Montekiego
FILE 2 | LINE 24 |  PIOTR
FILE 1 | LINE 26 |  * PIOTR
FILE 2 | LINE 28 |  * JULIA córka Kapuletów
FILE 1 | LINE 30 |  * JULIA — córka Kapuletów
FILE 2 | LINE 30 |  * Obywatele weroneńscy, różne osoby płci obojej, liczący się przyjaciół obu domów, straż wojskowa i inne osoby.
FILE 1 | LINE 32 |  * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, wojskowa i inne osoby.
FILE 2 | LINE 35 | Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu w Mantui.
FILE 1 | LINE 37 | Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu w Mantui.PROLOG
FILE 1 | LINE 39 | Jan Kasprowicz
FILE 2 | LINE 38 | 
FILE 2 | LINE 39 | PROLOG
FILE 2 | LINE 40 | 
FILE 2 | LINE 41 | Przełożył Jan Kasprowicz